In [2]:
import duckdb
import pandas as pd
import xml.etree.ElementTree as ET

In [3]:
#tabla paises 
pais = duckdb.sql('''SELECT name AS Nombre FROM enunciado_paises.csv''')
df_pais = pais.df()
df_pais

,Nombre
0,Belgium
1,England
2,France
3,Germany
4,Italy
5,Netherlands
6,Poland
7,Portugal
8,Scotland
9,Spain


In [4]:
#tabla jugador
jugador = duckdb.sql('''SELECT player_api_id AS ID, player_name AS Nombre FROM enunciado_jugadores.csv''')
df_jugador = jugador.df()
df_jugador

,ID,Nombre
0,505942,Aaron Appindangoye
1,155782,Aaron Cresswell
2,162549,Aaron Doran
3,30572,Aaron Galindo
4,23780,Aaron Hughes
...,...,...
11055,26357,Zoumana Camara
11056,111182,Zsolt Laczko
11057,36491,Zsolt Low
11058,35506,Zurab Khizanishvili


In [24]:
#tabla temporada, el id fue creado con pandas
temporada = duckdb.sql('''SELECT DISTINCT season AS Temporada FROM enunciado_partidos.csv''')
df_temporada = temporada.df()         # paso el objeto duckdb a df de pandas
df_temporada['ID'] = range(len(df_temporada),0,-1) # para cada valor de temporada en la nueva columna pongo ese id
df_temporada

,Temporada,Id_temporada
0,2009/2010,8
1,2012/2013,7
2,2011/2012,6
3,2015/2016,5
4,2013/2014,4
5,2008/2009,3
6,2014/2015,2
7,2010/2011,1


In [5]:
# tabla liga, EL ID ESTA MAL, debería ser uno de estos [1,  1729,  4769,  7809, 10257, 13274, 15722, 17642, 19694,21518, 24558]
# pero como saber cual es cual? YA SE, gracias al country_id del partido
liga = duckdb.sql('''
                  SELECT DISTINCT l.name AS Nombre_liga, p.name AS Nombre_pais, part.league_id AS ID
                  FROM enunciado_liga.csv AS l
                  INNER JOIN enunciado_paises.csv AS p ON l.country_id = p.id
                  INNER JOIN enunciado_partidos.csv AS part ON l.country_id = part.country_id
                  ''')

df_liga = liga.df()
df_liga


,Nombre_liga,Nombre_pais,ID
0,Netherlands Eredivisie,Netherlands,13274
1,Belgium Jupiler League,Belgium,1
2,Spain LIGA BBVA,Spain,21518
3,Scotland Premier League,Scotland,19694
4,England Premier League,England,1729
5,Italy Serie A,Italy,10257
6,Portugal Liga ZON Sagres,Portugal,17642
7,France Ligue 1,France,4769
8,Germany 1. Bundesliga,Germany,7809
9,Switzerland Super League,Switzerland,24558


In [6]:
# Tabla Equipos
equipo = duckdb.sql('''
                    SELECT DISTINCT eq.team_api_id AS ID_equipo, eq.team_long_name AS Nombre, par.league_id AS ID_liga
                    FROM enunciado_equipos.csv AS eq
                    INNER JOIN enunciado_partidos.csv AS par ON eq.team_api_id = par.home_team_api_id
                    ''')

df_equipo = equipo.df()
#EXPLICACION: basicamente hice el join en base a la liga donde ese equipo jugó de local, podria pasar que no hay jugado de local?
# según lo que estuve viendo no, porque me unió a todos los equipos
df_equipo


,ID_equipo,Nombre,ID_liga
0,9879,Fulham,1729
1,10261,Newcastle United,1729
2,9798,Reading,1729
3,8659,West Bromwich Albion,1729
4,8472,Sunderland,1729
...,...,...,...
294,8571,KV Kortrijk,1
295,8462,Portsmouth,1729
296,7730,FC Lausanne-Sports,24558
297,10268,Elche CF,21518


In [5]:
goleador = duckdb.sql('''
                      SELECT DISTINCT match_api_id AS ID_partido, goal AS Goles
                      FROM enunciado_partidos.csv
                      ''')

df_goleador = goleador.df().dropna(subset=['Goles'])
goles = df_goleador["Goles"]
#print(f"Hay {len(goles)} filas")
columna_final = []

for gol in goles:
    # Parsear el string XML
    root = ET.fromstring(gol)
    # Función para convertir XML a diccionario
    def xml_to_dict(element):
        result = {}
        if element.tag == "goal":
            result["values"] = []  # Inicializa la lista de valores
            for value in element.findall('value'):
                value_dict = xml_to_dict(value)  # Convierte cada valor a dict
                result["values"].append(value_dict)  # Agrega el dict a la lista
        else:
            for child in element:
                # Llamar recursivamente si hay más hijos
                result[child.tag] = xml_to_dict(child) if len(child) > 0 else child.text
        return result
    # Convertir el XML a diccionario
    goal_dict = xml_to_dict(root)
    # Appendeamos
    columna_final.append(goal_dict)
    



[{'values': [{'comment': 'n',
    'stats': {'goals': '1', 'shoton': '1'},
    'event_incident_typefk': '393',
    'coordinates': {'value': '6'},
    'elapsed': '38',
    'player2': '103899',
    'subtype': 'shot',
    'player1': '30929',
    'sortorder': '1',
    'team': '10261',
    'id': '5120237',
    'n': '312',
    'type': 'goal',
    'goal_type': 'n'},
   {'comment': 'n',
    'stats': {'goals': '1', 'shoton': '1'},
    'event_incident_typefk': '393',
    'coordinates': {'value': '60'},
    'elapsed': '61',
    'player2': '261313',
    'subtype': 'shot',
    'player1': '154280',
    'sortorder': '5',
    'team': '10252',
    'id': '5120635',
    'n': '483',
    'type': 'goal',
    'goal_type': 'n'}]},
 {'values': [{'comment': 'n',
    'stats': {'goals': '1', 'shoton': '1'},
    'event_incident_typefk': '131',
    'coordinates': {'value': '2'},
    'elapsed': '3',
    'subtype': 'loose_ball',
    'player1': '417068',
    'sortorder': '2',
    'team': '9817',
    'id': '5123883',
  

In [ ]:
for goles_en_partido in columna_final:
    for gol in goles_en_partido['values']:

        print(gol.keys())
        gol['player1'] # id_jugador
        print(gol['stats'])
        if gol['stats']['goals'] != 1:
            print(f"Hay {gol['stats']['goals']} goles en una llamada.")